# Analyzing and Visualizing WWII Aerial Bombing Data 
## By Justin McKinney and Justin Ashbaugh

<left><img src="images/formation.jpg" alt="what image shows" height="600" width="1000"></left>

## Introduction
World war II has been sealed in the history books as one of the most catastrophic, devastation-causing events in all of history.  Resulting in billions of dollars worth of damages and casualties in the millions, there was no shortage of pure carnage to be seen. Though these numbers are clear and present, it is sometimes important to ponder what it takes for sheer destruction of this magnitude.
The data set we analyze here houses a plethora of knowledge and data regarding the destructive capabilities of aircraft during WWII. It houses many important data points such as Aircraft type, Bomb type, target, bomb size, and so much more. It is important to note, however, that this data only represents targets and missions conducted by the allied forces, without the inclusion of Russia. This data is not representative of the axis powers' contribution to the world's destruction.
### Outline
In this tutorial, our goal is to provide you with a more colloquial and readable understanding of the historical data collected during the war. The data provided in its raw form is incredibly clunky, laden with missing values and column after column of confusing information. Our tutorial works to tidy this data, as well as to manipulate it and frame it for better understanding and investigation. We hope that after reading our tutorial you walk away with a better understanding of the destruction of WWII, and the sheer amount of bombs that were dropped as a result of the multiple theatres of war. Perhaps with this knowledge, you can hold an engaging conversation with your friends, state a fun fact or two, or even write a paper about the tragedy and horror of human ingenuity. If you already knew all of this information, we hope you enjoyed our visual representations and a short refresher course in destructive history. If this is new to you, rejoice in the fact that we live in a world where this war has ended, and that you need not fear the looming threat of a B17 bomber overhead.
#### A few things to note
- Here is the data set source: https://data.world/datamil/world-war-ii-thor-data
- This is real historical data so there is a LOT of missing data, our default behavior is to ignore missing data but note that it will skew our results to some degree
- This dataset does not include any Russian data nor any Axis bombing data
#### Brief Explanation of libraries used
- [pandas](https://pandas.pydata.org/docs/) to hold and represent data
- [numpy](https://numpy.org/doc/stable/) for various numerical conveniences
- [seaborn](https://seaborn.pydata.org/tutorial.html) for pretty graphs and visuals
- [matplotlib](https://matplotlib.org/stable/api/index) for working with seaborns graphs
- [folium](http://python-visualization.github.io/folium/) for doing heatmaps
- [sklearn](https://scikit-learn.org/stable/user_guide.html) for machine learning

## Step 1: Prep the Dataset

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap
from sklearn import tree
from sklearn import metrics
from sklearn.model_selection import *
import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv("data/THOR_WWII_DATA_CLEAN.csv", sep=",", encoding="latin-1") # For some reason the CSV did not like to be loaded without specifiying this encoding
data

## Tidy the data
Handling missing data is one of the key elements of data science and manipulation/analysis. 
In our case, since this is historical data, a lot of information is missing. Lucky for us most of that information
would not be useful for our analysis anyway. 
Let's drop some of those unnecessary columns that we won't be using to make the data a bit more readable.


In [ ]:
#drop a large amount of columns, mostly filled with NAN or irrelevent information
data.drop(['CALLSIGN','ROUNDS_AMMO','SPARES_RETURN_AC','WX_FAIL_AC','MECH_FAIL_AC','MISC_FAIL_AC','TARGET_COMMENT','MISSION_COMMENTS','SOURCE','DATABASE_EDIT_COMMENTS'], axis=1, inplace=True)
data.drop(['AC_LOST','AC_DAMAGED','AC_AIRBORNE','AC_DROPPING','TIME_OVER_TARGET','SIGHTING_METHOD_CODE','SIGHTING_EXPLANATION','BDA'], axis=1, inplace=True)
data.drop(['THEATER','NAF','TGT_COUNTRY_CODE','NUMBER_OF_FRAG','TYPE_OF_FRAG','LBS_FRAG','TONS_OF_FRAG','TOTAL_LBS'], axis=1, inplace=True)
data.drop(['MASTER_INDEX_NUMBER','WWII_ID','TGT_ID','TGT_INDUSTRY_CODE','TGT_INDUSTRY','LBS_HE','TONS_OF_HE','NUMBER_OF_IC','TYPE_OF_IC','LBS_IC','TONS_OF_IC'], axis=1, inplace=True)
data.drop(['UNIT_ID','TGT_PRIORITY_EXPLANATION','ALTITUDE','ALTITUDE_FEET','NUMBER_OF_HE','TYPE_OF_HE'], axis=1, inplace=True)
data.drop(['MDS','MSN_TYPE','TGT_PRIORITY','SOURCE_LATITUDE', 'SOURCE_LONGITUDE'], axis=1, inplace=True)
data

## Step 2: Analysis
Now that we have loaded the data let's do some basic analysis!  Let's try to figure out how much bombing the whole data set recorded.  We can do this easily by specifying the column containing the total tons of weaponry dropped and calling the sum on that column.  We are going to use the TOTAL_TONS column which expresses the amount of bombs dropped by the explosive energy they produce in [tons of TNT](https://en.wikipedia.org/wiki/TNT_equivalent). This is useful as it allows us to not overcount lots of very small bombs being dropped frequently nor undercount any large payloads dropped infrequently.

In [ ]:
#This is a simple pandas call that collects the column "TOTAL_TONS from the dataframe and computes the sum of all values"
total_tons = data["TOTAL_TONS"].sum() 
total_tons

That is a pretty insane amount of weaponry dropped throughout WW2.  Now let's try to recreate that graph on the website we got the data from that showed which countries had the most tons of weapons dropped on them. Graphs are an incredibly valuable source of information and visualization when analyzing data. It is much easier to analyze data when we have a graphical representation 
of its magnitude, and ability to gauge a more relative size with respect to other information.


In [ ]:
# We want to group the data by country being bombed, sum the total tons dropped, and sort by decreasing order.  Then we to reset
# the index so we can query by TGT_COUNTRY and then grab the first 5 elements to graph
per_country = (data.groupby(["TGT_COUNTRY"]).sum()).sort_values(by="TOTAL_TONS", ascending=False).reset_index().head(5)

plt.figure(figsize=(20,10)) # Set image size
bar_graph = sns.barplot(x="TGT_COUNTRY",y="TOTAL_TONS", data=per_country)
# This line makes it so we don't use scientific notation for the y axis
bar_graph.ticklabel_format(style="plain", axis="y")
# Set labels
bar_graph.set_xlabel("Target Country")
bar_graph.set_ylabel("Tons of TNT")
bar_graph.set_title("Total bombing by payload size")


Wow! That is a lot of bombs dropped on Germany.  Now let's see who was dropping those bombs.  We can do a similar strategy as the last graph except for this time we will group by the country flying the mission instead of the target country. Knowing that the United States and Britain were the main actors on the allied side of WWII, we can expect them to have the biggest chunks of the graph!Wow! That is a lot of bombs dropped on Germany.  Now let's see who was dropping those bombs.  We can do a similar strategy as the last graph except for this time we will group by the country flying the mission instead of the target country.

In [ ]:
#Call to pandas groupby function which groups the data by a specified column
most_dropped = data.groupby(["COUNTRY_FLYING_MISSION"]).sum().sort_values(by=["TOTAL_TONS"],ascending=False).reset_index()
plt.figure(figsize=(20,10))
#Here we use the seaborn barplot function. We have found seaborn to be very user friendly and easy to understand!
bar_graph = sns.barplot(x="COUNTRY_FLYING_MISSION",y="TOTAL_TONS", data=most_dropped)
bar_graph.ticklabel_format(style="plain", axis="y")
bar_graph.set_xlabel("Country dropping")
bar_graph.set_ylabel("Tons of TNT")
bar_graph.set_title("Tons of TNT dropped by each country")

The USA and Great Britain unsurprisingly did the most dropping of bombs.  Now let's find which planes did most of this bomb dropping.

In [ ]:
# Lets group the data by aircraft name and see which aircraft has dropped the most tons of bombs
aircrafts = data.groupby(["AIRCRAFT_NAME"]).sum().sort_values(by=["TOTAL_TONS"],ascending=False).reset_index().head(9) # Lets get the top 9 and leave room for an other category
# Create percentage col to use for piechart
aircrafts["PERCENTAGE_DROPPED"] = (aircrafts["TOTAL_TONS"]/total_tons)*100
# Calculate remaining
other_percentage = 100 - aircrafts["PERCENTAGE_DROPPED"].sum()
# Append other to the dataframe
aircrafts = aircrafts.append({"PERCENTAGE_DROPPED" : other_percentage,"AIRCRAFT_NAME" : "Other" }, ignore_index=True)
# plot
plt.figure(figsize=(20,10))
colors = sns.color_palette()[0:10]
plt.pie(aircrafts["PERCENTAGE_DROPPED"], colors=colors, labels=aircrafts["AIRCRAFT_NAME"], autopct="%.1f%%")
plt.title("Percentage of all bombs dropped by aircraft")
plt.show()


The [B17](https://en.wikipedia.org/wiki/Boeing_B-17_Flying_Fortress) leads this category having dropped almost 30% of all bombs dropped by the US and Great Britain over the war.

![B17](images/b17.jpg)

This is impressive but not unexpected as the B17 was one of the most mass-produced and effective bombers of the war.  Britannica states that the B17 ["was the mainstay of the strategic bombing campaign"](https://www.britannica.com/technology/B-17) for the US.

Now let's try to get a nice overview of the amount of bombing that occurred over time.

In [ ]:
data["DATE_TIME"] = pd.to_datetime(data["MSNDATE"], format="%m/%d/%Y") # Convert to datetimes for ease of use
by_time = data.groupby(["DATE_TIME"]).sum().reset_index()
plt.figure(figsize=(20,10))
line_graph = sns.lineplot(x="DATE_TIME",y="TOTAL_TONS", data=by_time)
line_graph.set_xlabel("Year")
line_graph.set_ylabel("Tons of TNT")
line_graph.set_title("Tons of TNT over time")

Hmm, what is that slight blip we see right before 1941?  That seems to be a lot of bombing for so early in the war.

In [ ]:
before_1941 = data[data["DATE_TIME"].dt.year < 1941]
outlier = before_1941[before_1941["TOTAL_TONS"] > 4000]
outlier[["TGT_COUNTRY","AIRCRAFT_NAME", "TOTAL_TONS", "TGT_TYPE", "DATE_TIME","AC_ATTACKING"]]

It turns out this occurred in Africa and was related to [Northern front, East Africa, 1940](https://en.wikipedia.org/wiki/Northern_front,_East_Africa,_1940).  Our data seems to correlate with the Wikipedia article under the section about the British attack on fort Gallabat.  Our data shows 6 WELLESLEY bombers attacking a fort with around 5000 tons of TNT worth of bombs.  Wikipedia states "An RAF contingent of six Wellesley bombers and nine Gladiator fighters were thought sufficient to overcome the 17 Italian fighters and 32 bombers believed to be in range. The infantry assembled 1–2 mi (2–3 km) from Gallabat, whose garrison was unaware that an attack was coming until the RAF bombed the fort and put the wireless out of action.".  An interesting little discovery.

This next section of code works with a slightly more advanced understanding of plotting and data frame manipulation in pandas. The goal of this cell is to identify a list of all aircraft used during WWII, creating and labeling a scatter plot which will be used to display the amount of TNT (in tons) dropped by each aircraft type, throughout the war. We utilize a clever [subplot](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html) stacking trick to put multiple plots on the same single plane(pun intended). We then loop through our list of planes, for each plane isolating our data frame to only contain those rows that relate to the current plane, and for these rows plotting their data points on our graph.

In [ ]:
#Create a list of plane names
planes = aircrafts.loc[:,'AIRCRAFT_NAME']
#create a plot object
fig = plt.figure(figsize = (20,10))
#label the plot
plt.title("Tons of Bombs dropped by Aircraft With Respect to time")
plt.xlabel("Year")
plt.ylabel("Tons of TNT")
#utilize subplot stacking to create multiple planes for multiple plots
stack = fig.add_subplot(111)
#iterate over list of planes, plotting respective year and total tons of TNT per plane
for i in planes:
    plot_table = data[data["AIRCRAFT_NAME"]==i]
    plot_table = plot_table.sort_values(by=['DATE_TIME'])
    stack.scatter(plot_table['DATE_TIME'],plot_table['TOTAL_TONS'])
#create legend for list of planes
stack.legend(planes)

Upon first view of the above graph, it might seem like it was done incorrectly, and the code does not work the way it is supposed to. This was our initial thought when seeing the clustered results followed by two major outliers far above any other data points. We decided to pull up those particular data points from our data frame to see what was going on, as these data points made no sense. The following code is what we used to learn more about our very large outliers and determine what our issue could be.

In [ ]:
#Isolate the data which appears to be above 14000 total tons in our graph
big = data[data['TOTAL_TONS'] > 14000]
#clean up our dataframe to data that is only relevent to our interpretation
big = big[['COUNTRY_FLYING_MISSION','TGT_COUNTRY','TGT_LOCATION','TGT_TYPE','TOTAL_TONS']]
#print dataframe for visualization
big

After successfully isolating the two outliers we were able to determine that there was, in fact, no bug in our code, but an anomaly 
regarding the two and only atomic bombs ever used on civilization in all of history. We were surprised to have forgotten these two events, though
they immediately made the data clear and understandable. The events, as listed in the data frame correlate to the bombings of [Hiroshima](https://en.wikipedia.org/wiki/Hiroshima#World_War_II_and_the_atomic_bombing_(1939%E2%80%931945)) and [Nagasaki](https://en.wikipedia.org/wiki/Nagasaki#Atomic_bombing_of_Nagasaki_during_World_War_II)
by the united states, in August of 1945, using the catastrophic destruction of the atomic bomb. Bombs [Fat man](https://en.wikipedia.org/wiki/Fat_Man), and [Little Boy](https://en.wikipedia.org/wiki/Little_Boy) were dropped, with Fat man being the larger of the two. 
The Bombs can be seen Below. (Little Boy seen first, Fat Man seen second). Next to them is the b29, the bomber that carried them.


![Fat Man](images/fatman.jpg)
![Little Boy](images/littleboy.jpg)
![b29](images/b29.jpg)

## Step 3: More in depth visualization
Now let's bring in folium to do some visualizing of what areas were bombed the most.  Folium will let us create an interactive heatmap to see what areas were most bombed by the allies throughout the war.  We are going to do a frequency heatmap overlayed with circle markers that denote the most intensive explosions (top 10,000).  These circles will indicate, not to scale, the amount of damage caused in relation to other strikes.  We also will label a few key cities to guide your interpretation.

In [ ]:
# Important cities LATITUDE, LONGITUDE
berlin = 52.520008,13.404954
tokyo = 35.652832, 139.839478
rome = 41.902782, 12.496366
# Atomic Bombs
Hiroshima = 34.3853, 132.4553
Nagasaki = 32.7503, 129.8779

map_osm = folium.Map(location=berlin, zoom_start=5) #Create a map centered on Berlin

valid_lat = data.groupby(data["LATITUDE"].isnull()).get_group(False) # Make sure latitude is not NaN
valid_lat_long = valid_lat.groupby(data["LONGITUDE"].isnull()).get_group(False) # Make sure longitutde is not NaN
valid = valid_lat_long.groupby(data["TOTAL_TONS"].isnull()).get_group(False) # Make sure total tons is not NaN

heat_map_data = []
# Iterate through all rows in Ton order and add the relevant info to the map
for index, row in valid.sort_values(by="TOTAL_TONS", ascending=False).reset_index().iterrows():
    if index < 10000: # Plot the top 10,000 with circles
        folium.Circle(
            location=[row["LATITUDE"],row["LONGITUDE"]],
            radius=row["TOTAL_TONS"]*2, # Adding a bit of scaling here to make bomb sizes distinguishable
            color="crimson",
            fill=True
        ).add_to(map_osm)
    # Add to heatmap data
    heat_map_data.append([row["LATITUDE"],row["LONGITUDE"]])
#Specify heatmap color gradients
colormap = {0.3: 'white', 0.6: 'grey', 1: 'black'}
# Add the list of points to the map as a heatmap
HeatMap(heat_map_data,gradient=colormap).add_to(map_osm) 
# Label the capital Axis cities
folium.Marker(location=berlin,popup="<b>Berlin</b>").add_to(map_osm) 
folium.Marker(location=tokyo,popup="<b>Tokyo</b>").add_to(map_osm)
folium.Marker(location=rome,popup="<b>Rome</b>").add_to(map_osm)
# Label nuclear bomb sites
folium.Marker(location=Hiroshima,popup="<b>Hiroshima</b>").add_to(map_osm)
folium.Marker(location=Nagasaki,popup="<b>Nagasaki</b>").add_to(map_osm)
# Show map
map_osm

    

## Step 4: A bit of machine learning (because everything needs machine learning)
Let's say hypothetically you are a person living your life during WW2.  We are going to train a machine learning model to predict which type of aircraft would be most likely to drop a bomb on your head.  This is very tongue in cheek but an interesting way to learn about some basic machine learning concepts.
### Details
We are going to train a [Decision Tree](https://en.wikipedia.org/wiki/Decision_tree_learning) model in order to predict aircraft type based on given longitude and latitude.  We will split the data into a training set and a testing set, then train the model on the training set.  After that, we can use the testing set to test how accurate our model is.  This is called [holdout validation](https://www.datarobot.com/wiki/training-validation-holdout/) and is a common technique in machine learning.

In [ ]:
valid = valid_lat_long.groupby(data["AIRCRAFT_NAME"].isnull()).get_group(False) # Make sure lattitude, longitude, and aircraft name are not missing
SEED = 5 # Random number seed to make results reproduceable
plane_type = []
coordinates = []
for index, row in valid.iterrows():
    coordinates.append([row["LATITUDE"],row["LONGITUDE"]])
    plane_type.append(row["AIRCRAFT_NAME"])

X = coordinates
y = plane_type
#Split the data up
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size= 0.75,test_size=0.25,random_state=SEED)
# Train model
tree_model = tree.DecisionTreeClassifier(splitter="best",max_features = "auto", random_state=SEED).fit(X_train,y_train)
# Run a prediction on the test data
tree_predicted_y = tree_model.predict(X_test)
# Evaluate model
print(metrics.classification_report(y_test, tree_predicted_y))

This is not an intelligent application of machine learning but it does seem to produce not wildly incorrect results with an accuracy score of 0.58 out of 1.  To make a truly accurate model we would need way more data and include far more factors besides just latitude and longitude.  A decision tree also might not be the best model for the relationship we are trying to show.  Obviously trying to predict what aircraft would bomb a specific location is an impossible task but it is amusing to see what the model guesses for any longitude and latitude you throw at it.  One good takeaway lesson to be seen is the accuracy of the model by each plane.  Notice how the planes that were less common in the dataset have far lower [f1 scores](https://deepai.org/machine-learning-glossary-and-terms/f-score) meaning that they were predicted correctly far less than planes that were more common like the B29.  Planes like the Beaufighter or the F06 had very few entries so the model really struggled to properly predict them.  Also notice how the B17, the most common aircraft, seemed to have an oddly low score for being so common.  Compared to another very common bomber like the B29 it is odd that it has such a low f1 score.  This is likely because the B17 was used in so many more locations because of its long service span that the model had a hard time finding a proper correlation between location and where the B17 was used.

Now for fun let's try a few tests:

In [ ]:
tokyo = [35.6762,139.6503]
brussles = [50.8476, 4.3572]
rome = [41.902782, 12.496366]
singapore = [1.290270, 103.851959]
print(f"Prediction for Tokyo {tree_model.predict([tokyo])}")
print(f"Prediction for Brussels {tree_model.predict([brussles])}")
print(f"Prediction for Rome {tree_model.predict([rome])}")
print(f"Prediction for Singapore {tree_model.predict([singapore])}")

Some of these predictions do seem reasonable.  Tokyo for example is logical as the US did not bomb Japan until late into the war and the B29 came into service much later in WW2.  Brussels also makes some sense because B17's were mostly deployed over Europe.  Not all make complete sense but it is interesting to see how the inputs impact the prediction.

# Conclusion
<left><img src="images/end.jpg" alt="what image shows" height="600" width="1000"></left>

"[Victory in the Pacific, Soldiers Celebrate Peace](https://www.historynet.com/v-j-day-1945-the-world-rejoices.htm)"


In this tutorial, we set out to inform you, the reader, about the intricacies of the data science pipeline with a fun, informative, and entertaining series of code snippets and explanations, 
that leave you wanting to explore deeper into the world of data science. We hope the destructive intricacies of WWII bombings were able to keep you interested and engaged with what we had to offer. From tidying/wrangling the expansive amounts of data to data analysis with heatmaps and folium, and even to small-scale machine learning models, we hope you have learned at least some of the basics of what it takes to be a data scientist. The data analysis we have conducted allows us to achieve a better understanding of what sheer destructive power we as human beings are capable of reaching. 
Despite the somewhat intriguing and perhaps even cool nature of this data... It is important to remember that real people experienced a world in which these numbers were more than just numbers, a cruel reality where life had extreme uncertainty, and nowhere was truly safe.  It is our duty as a collective and united species to work together on our problems and to never reach hostilities of any degree similar to that of WWII. Surely this would lead to total annihilation.